### Übungen

#### Einlesen der Daten

Einlesen der Daten in eine Datenstruktur, auf die wir mit Koordinaten zugreifen können.

In [4]:
%%writefile input.txt
4 8
######## 
#   #  #
#  ##  #
########


Overwriting input.txt


In [5]:
def show(grid):
    for zeile in grid:
        print(*zeile,sep='')

f = open('input.txt')
hoehe, breite = [int(x) for x in f.readline().split()]   # unpacking
print(f'hoehe={hoehe}, breite={breite}')
grid = []
for k in range(hoehe):
    grid.append(list(f.readline().strip()))
f.close()

show(grid)



hoehe=4, breite=8
########
#   #  #
#  ##  #
########


In [8]:
# Syntax-Übung
a = [1,2,3,4]
b = [2*x for x in a if x%2 == 0]    # list-comprehension
b

[4, 8]

#### Aufbau des Graphen

Jeder begehbare Punkt ist ein Knoten. Die Koordinate ist der Name des Knoten

In [12]:
V = []
for x in range(hoehe):
    for y in range(breite):
        if grid[x][y] != '#':
            V.append((x,y))
V

[(1, 1), (1, 2), (1, 3), (1, 5), (1, 6), (2, 1), (2, 2), (2, 5), (2, 6)]

In [15]:
# dasselbe wie oben mit einer list-comprehension
V = [(x,y) for x in range(hoehe) for y in range(breite) if grid[x][y] != '#']   # Liste mit Knoten
V

[(1, 1), (1, 2), (1, 3), (1, 5), (1, 6), (2, 1), (2, 2), (2, 5), (2, 6)]

In [16]:
G = {v:set() for v in V}              # noch haben die Knoten keine Nachbarn
dirs = [(0,1),(1,0),(0,-1),(-1,0)]    # Richtungen O,S,W,N

for v in G:
    x, y = v
    for (xd,yd) in dirs:
        xn, yn = x+xd, y+yd
        if 0 <= xn < hoehe and 0 <= yn < breite and grid[xn][yn] != '#':
            G[v].add((xn,yn))
G

{(1, 1): {(1, 2), (2, 1)},
 (1, 2): {(1, 1), (1, 3), (2, 2)},
 (1, 3): {(1, 2)},
 (1, 5): {(1, 6), (2, 5)},
 (1, 6): {(1, 5), (2, 6)},
 (2, 1): {(1, 1), (2, 2)},
 (2, 2): {(1, 2), (2, 1)},
 (2, 5): {(1, 5), (2, 6)},
 (2, 6): {(1, 6), (2, 5)}}

#### Erreichbarkeit (dfs)

In [23]:
def dfs(v):  
    visited[v] = True
    for w in G[v]:
        if not visited[w]:
            dfs(w) 

visited =  {v : False for v in G}    
dfs((1,6))
erreichbar = [v for v in G if visited[v]]
erreichbar

[(1, 5), (1, 6), (2, 5), (2, 6)]

#### Zusammenhangskomponenten (connected components)

Wir zählen die getrennten Räume im grid, indem wir von jedem noch nicht besuchten Knoten ein dfs starten.

In [26]:
def dfs(v):  
    visited[v] = True
    for w in G[v]:
        if not visited[w]:
            dfs(w) 
            
visited =  {v : False for v in G}   
zaehl = 0
for v in G:
    if not visited[v]:
        zaehl+=1
        dfs(v)

print(f'Es gibt {zaehl} Räume.')

Es gibt 2 Räume.


#### Kürzeste Wege mit Breitensuche (bfs)

Overwriting input.txt


In [82]:
%%writefile input.txt
5 6
######
#A   #
#### #
#B   #
######

Overwriting input.txt


Einlesen der Daten

In [83]:
f = open('input.txt')
hoehe, breite = [int(x) for x in f.readline().split()]   # unpacking
print(f'hoehe={hoehe}, breite={breite}')
grid = []
for k in range(hoehe):
    grid.append(list(f.readline().strip()))
f.close()
show(grid)

hoehe=5, breite=6
######
#A   #
#### #
#B   #
######


Aufbau des Graphen

In [84]:
V = [(x,y) for x in range(hoehe) for y in range(breite) if grid[x][y] != '#']
G = {v:set() for v in V}
dirs = [(0,1),(1,0),(0,-1),(-1,0)]    # Richtungen O,S,W,N

for v in G:
    x, y = v
    for (xd,yd) in dirs:
        xn, yn = x+xd, y+yd
        if 0 <= xn < hoehe and 0 <= yn < breite and grid[xn][yn] != '#':
            G[v].add((xn,yn))

Bestimmung von Start- und Zielkoordinaten

In [85]:
for x in range(hoehe):
    for y in range(breite):
        if grid[x][y] == 'A':
            start = (x,y)
        elif grid[x][y] == 'B':
            ziel = (x,y)

print(f'start={start}, ziel={ziel}')

start=(1, 1), ziel=(3, 1)


#### Breitensuche

Die Breitensuche speichert ihre Ergebnisse in den dictionaries *prev* und *dist*.

In dem dictionary *dist* speichern wir die Distanz eines Knotens zum Startknoten. Wenn die Distanz unendlich (*inf*) ist, dann haben wir den Knoten noch nicht besucht. In dem dictionary *prev* wird der Vorgänger des Knotens gespeichert. *prev* modelliert den shortest-path-Baum der Breitensuche.

In [87]:
from collections import deque
inf = float('inf')
dist = {v:inf for v in G}
prev = {v:None for v in G}

dist[start] = 0       # s ist der Startknoten
Q = deque([start])    # Zu Beginn ist nur der Startknoten in der Schlange       

while Q:                           # Solange die Schlange nicht leer      
    u = Q.popleft()                # hole das erste Element u
    for v in G[u]:                 # und kümmere dich darum, d.h. für alle Nachbarn v von u
        if dist[v] == inf:         # falls Nachbar v noch nicht besucht wurde
            dist[v] = dist[u]+1    # zu v benötigen wir einen Schritt mehr als zu u
            prev[v] = u            # Vorgänger von v ist u
            Q.append(v)            # füge v in die Schlange ein (damit später dessen Nachbarn betrachtet werden)


In [88]:
print(prev)
print(dist)

{(1, 1): None, (1, 2): (1, 1), (1, 3): (1, 2), (1, 4): (1, 3), (2, 4): (1, 4), (3, 1): (3, 2), (3, 2): (3, 3), (3, 3): (3, 4), (3, 4): (2, 4)}
{(1, 1): 0, (1, 2): 1, (1, 3): 2, (1, 4): 3, (2, 4): 4, (3, 1): 8, (3, 2): 7, (3, 3): 6, (3, 4): 5}


Aus dem dict prev rekonstruieren wir den Pfad des kürzesten Weges.

In [89]:
def reconstructPath(start,ziel,prev):
    result = []                  # hier sammeln wir die Vorgängerknoten
    while ziel != start:         # solange wir mit ziel noch nicht bei start angekommen sind
        result.append(ziel)      # füge zielknoten in die result-liste ein
        ziel = prev[ziel]        # neues ziel ist der vorgänger von ziel
    result.append(start)         # als letzes fügen wir den startknoten in die result-liste ein
    result.reverse()             # wir drehen die Reihenfolge um
    return result 

path = reconstructPath(start,ziel,prev)
path

[(1, 1), (1, 2), (1, 3), (1, 4), (2, 4), (3, 4), (3, 3), (3, 2), (3, 1)]

Wir markieren den gefunden Pfad in dem grid. Den Start und Zielknoten wollen wir nicht überschrieben.

In [90]:
path = reconstructPath(start,ziel,prev)
for v in path[1:-1]:                # alles außer ersten und letzten Knoten
    x, y = v
    grid[x][y] = '.'

print(f'Länge des kürzesten Weges: {len(path)-1}')
show(grid)

Länge des kürzesten Weges: 8
######
#A...#
####.#
#B...#
######


Der gesamte Ablauf der Breitensuche:

In [91]:
%%writefile input.txt
20 67
###################################################################
#                                                                 #
#                                                                 #
#                                                                 #
#                       ####################################      #
#                       #                                         #
#                       #                     #                   #
#                       #                     #                   #
#                       #                     #                   #
#                       #        B                     #          #
#                       #      ####################    #          #
#                       #                              #          #
#                       #                              #          #
#                       ################################          #
#                                                                 #
#                                                                 #
#                                                                 #
#     A                                                           #
#                                                                 #
###################################################################

Overwriting input.txt


In [95]:
# Einlesen der Daten
f = open('input.txt')
hoehe, breite = [int(x) for x in f.readline().split()]   # unpacking
print(f'hoehe={hoehe}, breite={breite}')
grid = []
for k in range(hoehe):
    grid.append(list(f.readline().strip()))
f.close()

V = [(x,y) for x in range(hoehe) for y in range(breite) if grid[x][y] != '#']
G = {v:set() for v in V}
dirs = [(0,1),(1,0),(0,-1),(-1,0)]    # Richtungen O,S,W,N

# Aufbau des Graphen
for v in G:
    x, y = v
    for (xd,yd) in dirs:
        xn, yn = x+xd, y+yd
        if 0 <= xn < hoehe and 0 <= yn < breite and grid[xn][yn] != '#':
            G[v].add((xn,yn))

# Bestimmung von Start und Zielknoten
for x in range(hoehe):
    for y in range(breite):
        if grid[x][y] == 'A':
            start = (x,y)
        elif grid[x][y] == 'B':
            ziel = (x,y)

print(f'start={start}, ziel={ziel}')

# Breitensuche
from collections import deque
inf = float('inf')
dist = {v:inf for v in G}
prev = {v:None for v in G}

dist[start] = 0       # s ist der Startknoten
Q = deque([start])    # Zu Beginn ist nur der Startknoten in der Schlange       

while Q:                           # Solange die Schlange nicht leer      
    u = Q.popleft()                # hole das erste Element u
    for v in G[u]:                 # und kümmere dich darum, d.h. für alle Nachbarn v von u
        if dist[v] == inf:         # falls Nachbar v noch nicht besucht wurde
            dist[v] = dist[u]+1    # zu v benötigen wir einen Schritt mehr als zu u
            prev[v] = u            # Vorgänger von v ist u
            Q.append(v)            # füge v in die Schlange ein (damit später dessen Nachbarn betrachtet werden)

# Rekonstruktion des Pfads
def reconstructPath(start,ziel,prev):
    result = []                  # hier sammeln wir die Vorgängerknoten
    while ziel != start:         # solange wir mit ziel noch nicht bei start angekommen sind
        result.append(ziel)      # füge zielknoten in die result-liste ein
        ziel = prev[ziel]        # neues ziel ist der vorgänger von ziel
    result.append(start)         # als letzes fügen wir den startknoten in die result-liste ein
    result.reverse()             # wir drehen die Reihenfolge um
    return result 


path = reconstructPath(start,ziel,prev)
print(f'Länge des kürzesten Weges: {len(path)-1}')
for v in path[1:-1]:                # alles außer ersten und letzten Knoten
    x, y = v
    grid[x][y] = '.'

show(grid)


hoehe=20, breite=67
start=(17, 6), ziel=(9, 33)
Länge des kürzesten Weges: 83
###################################################################
#                                                                 #
#                                                                 #
#                                                                 #
#                       ####################################      #
#                       #                                         #
#                       #                     #                   #
#                       #                     #                   #
#                       #                     #     .....         #
#                       #        B...................  #.         #
#                       #      ####################    #.         #
#                       #                              #.         #
#                       #                              #.         #
#                       ##############

### Dijkstra

Der Lageplan hat 2 Stockwerke: Obergeschoß und Erdgeschoß. Wir lesen zunächste beide Stockwerke getrennt ein.

In [97]:
%%writefile input.txt
5 13
#############
#A          #
#############           
#          B#
#############

#############
#     #     #
#### ########           
#     #     #
#############

Overwriting input.txt


In [106]:
f = open('input.txt')
hoehe, breite = [int(x) for x in f.readline().split()]
grid1 = []
for k in range(hoehe):
    grid1.append(list(f.readline().strip()))
f.readline()
grid2 = []
for k in range(hoehe):
    grid2.append(list(f.readline().strip()))
f.close()
 

In [107]:
print('Obergeschoß:')
show(grid1)
print('\nErdgeschoß:')
show(grid2)
 

Obergeschoß:
#############
#A          #
#############
#          B#
#############

Erdgeschoß:
#############
#     #     #
#### ########
#     #     #
#############


In [105]:
grid = [grid1, grid2]

Wir fügen die beiden grids zu einem 3-dimensionalen grid zusammen. Eine Koordinate besteht jetzt aus drei Zahlen. Die erste Zahl kennzeichnet das Stockwerk, die 2. Zahl die Zeile im Stockwerk, die 3. Zahl die Spalte.

In [113]:
print(grid[0][1][1]) 

A


Wir nutzen z als Variable für die erste Koordinate, damit wir mit x und y wie gewohnt über Zeile und Spalte laufen können.

In [133]:
# Liste mit den Knoten
V = [(z,x,y) for z in [0,1] for x in range(hoehe) for y in range(breite) if grid[z][x][y] != '#'] 
G = {v:dict() for v in V}

Die Liste *dirs* mit den Bewegungsrichtungen müssen wir erweitern, um die Stockwerke zu wechseln.

In [134]:
dirs = [(0,0,1),(0,1,0),(0,0,-1),(0,-1,0),(1,0,0),(-1,0,0)]    # Richtungen O,S,W,N,LOWER,HIGHER

In [135]:
for v in G:
    z, x, y = v
    for (zd, xd,yd) in dirs:
        zn, xn, yn = z+zd, x+xd, y+yd
        if 0<= zn <= 1 and 0 <= xn < hoehe and 0 <= yn < breite and grid[zn][xn][yn] != '#':
            if zn == z:     # wir bleiben in derselben Ebene
                G[v][(zn,xn,yn)] = 1
            else:           # wir wechseln das Stockwerk
                G[v][(zn,xn,yn)] = 3


In [136]:
print(G[(0,1,1)])

{(0, 1, 2): 1, (1, 1, 1): 3}


Wir bestimmen Start- und Zielknoten

In [137]:
for v in G:
    z,x,y = v
    if grid[z][x][y] == 'A':
        start = (z,x,y)
    elif grid[z][x][y] == 'B':
        ziel = (z,x,y)

print(f'start={start}, ziel={ziel}')

start=(0, 1, 1), ziel=(0, 3, 11)


In [153]:
from heapq import  heappop, heappush
inf = float('inf')
dist = {v:inf for v in G}
prev = {v:None for v in G}

 
dist[start] = 0
endgueltig = set()    

H = [(dist[v],v) for v in G]                       

while H:                                  # solange der Heap nicht leer
    _, u = heappop(H)                     # hole das erste Element u aus dem Heap  
    if u in endgueltig: continue          # falls es schon endgueltig markiert wurde, überspringen
    endgueltig.add(u)                     # für diesen Knoten u haben wir die kürzeste Distanz gefunden.
    for v in G[u]:                        # für jeden Nachbarn v von u:
        if dist[v] > dist[u] + G[u][v]:   # Falls der Weg zu v über u kürzer als der bislang gefundene Weg zu v                          
            dist[v] = dist[u] + G[u][v]   # speichere die kürzere Distanz für v
            prev[v] = u                   # speichere, dass diese Distanz über den Vorgänger u kommt
            heappush(H,(dist[v],v))    # speichere den Knoten mit der neuen Distanz im Heap

Aus dem dictionary *prev* rekonstruieren wir den kürzesten Weg

In [155]:
def reconstructPath(start,ziel,prev):
    result = []                  # hier sammeln wir die Vorgängerknoten
    while ziel != start:         # solange wir mit ziel noch nicht bei start angekommen sind
        result.append(ziel)      # füge zielknoten in die result-liste ein
        ziel = prev[ziel]        # neues ziel ist der vorgänger von ziel
    result.append(start)         # als letzes fügen wir den startknoten in die result-liste ein
    result.reverse()             # wir drehen die Reihenfolge um
    return result 


path = reconstructPath(start,ziel,prev)
print(f'Länge des kürzesten Weges: {len(path)-1}') 

Länge des kürzesten Weges: 14


Jetzt können wir den Weg markieren.

In [154]:
for v in path[1:-1]:                # alles außer ersten und letzten Knoten
    z, x, y = v
    grid[z][x][y] = '.'

show(grid[0])
print()
show(grid[1])

#############
#A...       #
#############
#    ......B#
#############

#############
#   . #     #
####.########
#   ..#     #
#############
